In [15]:
import os

import numpy as np
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
import pandas as pd

from sklearn import ensemble

#Load Data

In [11]:
#Load Data
FTRAIN = '~/kaggle/data/training.csv'
FTEST = '~/kaggle/data/test.csv'


def load(test=False, cols=None):
    """Loads data from FTEST if *test* is True, otherwise from FTRAIN.
    Pass a list of *cols* if you're only interested in a subset of the
    target columns.
    """
    fname = FTEST if test else FTRAIN
    df = read_csv(os.path.expanduser(fname))  # load pandas dataframe

    # The Image column has pixel values separated by space; convert
    # the values to numpy arrays:
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    if cols:  # get a subset of columns
        df = df[list(cols) + ['Image']]

    print(df.count())  # prints the number of values for each column
    df = df.dropna()  # drop all rows that have missing values in them

    X = np.vstack(df['Image'].values) / 255.  # scale pixel values to [0, 1]
    X = X.astype(np.float32)

    if not test:  # only FTRAIN has any target columns
        y = df[df.columns[:-1]].values
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        y = y.astype(np.float32)
    else:
        y = None

    return X, y


#X, y = load()
#print("X.shape == {}; X.min == {:.3f}; X.max == {:.3f}".format(
#    X.shape, X.min(), X.max()))
#print("y.shape == {}; y.min == {:.3f}; y.max == {:.3f}".format(
#    y.shape, y.min(), y.max()))


In [6]:
clf = ensemble.RandomForestClassifier()

In [8]:
clf.fit(X,y)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0)

In [12]:
xtest, dummy = load(test=True)
xtest

ImageId    1783
Image      1783
dtype: int64


array([[ 0.71372551,  0.71764708,  0.71372551, ...,  0.03137255,
         0.01960784,  0.01960784],
       [ 0.29803923,  0.34117648,  0.31764707, ...,  0.89411765,
         0.82745099,  0.65882355],
       [ 0.69411767,  0.6901961 ,  0.68235296, ...,  0.00392157,
         0.00392157,  0.00392157],
       ..., 
       [ 0.10980392,  0.10980392,  0.11372549, ...,  0.3019608 ,
         0.30588236,  0.30588236],
       [ 0.40784314,  0.37254903,  0.27843139, ...,  0.56470591,
         0.59215689,  0.62352943],
       [ 0.24705882,  0.23921569,  0.25098041, ...,  0.44313726,
         0.43921569,  0.44313726]], dtype=float32)

In [13]:
yhat_test = clf.predict(xtest)

In [14]:
yhat_test

array([[ 0.38252032, -0.20606992, -0.39504066, ...,  0.47307318,
        -0.01715447,  0.78947967],
       [ 0.41983333, -0.22288889, -0.42433333, ...,  0.52196294,
         0.02257407,  0.86127776],
       [ 0.3512654 , -0.27847368, -0.40403333, ...,  0.38601315,
        -0.25458911,  0.57708901],
       ..., 
       [ 0.35736486, -0.28235537, -0.45436001, ...,  0.33633497,
        -0.13182433,  0.57939678],
       [ 0.28946012, -0.28047854, -0.43058929, ...,  0.30625767,
        -0.09199026,  0.51712883],
       [ 0.22077477, -0.3180466 , -0.52348876, ...,  0.30925333,
        -0.18294805,  0.48739496]])

In [42]:
yhat_test_form = yhat_test*48 + 48

#Parse submission

In [20]:
idlook = '/Users/qcq/kaggle/data/IdLookupTable.csv'
sub = pd.DataFrame.from_csv(idlook)
sub.head(2)

,ImageId,FeatureName,Location
RowId,,,
1,1,left_eye_center_x,NaN
2,1,left_eye_center_y,NaN


In [21]:
cols = ['left_eye_center_x','left_eye_center_y','right_eye_center_x','right_eye_center_y','left_eye_inner_corner_x','left_eye_inner_corner_y','left_eye_outer_corner_x','left_eye_outer_corner_y','right_eye_inner_corner_x','right_eye_inner_corner_y','right_eye_outer_corner_x','right_eye_outer_corner_y','left_eyebrow_inner_end_x','left_eyebrow_inner_end_y','left_eyebrow_outer_end_x','left_eyebrow_outer_end_y','right_eyebrow_inner_end_x','right_eyebrow_inner_end_y','right_eyebrow_outer_end_x','right_eyebrow_outer_end_y','nose_tip_x','nose_tip_y','mouth_left_corner_x','mouth_left_corner_y','mouth_right_corner_x','mouth_right_corner_y','mouth_center_top_lip_x','mouth_center_top_lip_y','mouth_center_bottom_lip_x','mouth_center_bottom_lip_y','Image']

In [43]:
sub['Location'] = sub.apply(lambda row: yhat_test_form[row['ImageId']-1,cols.index(row['FeatureName'])],axis=1)

In [44]:
subname = '/Users/qcq/kaggle/data/sub20150524.csv'
sub['Location'].to_csv(subname,header=True)